In [1]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [118]:
#check list of files in data directory 
data_folder = './data/dedicated_host/'
print(os.listdir(folder))

['.DS_Store', 'm5d_pgbench_dedicated_host_11-16-2019.csv', 'm5d_y-cruncher_dedicated_host_11-16-2019_us-east-1 - Sheet1.csv', 'm5d_sysbench_dedicated_host_11-16-2019_us-west-1b - Sheet1.csv']


In [119]:
#make sure to import correct file here
data_name = 'm5d_pgbench_dedicated_host_11-16-2019.csv'

In [120]:
#adjust header if necessary
df_data = pd.read_csv(data_folder + data_name, header = 0)

In [121]:
#remove idle vm rows

#y-cruncher
#df_data.dropna(subset=['processorInfo'], inplace= True)

In [122]:
df_data.head()

,instanceID,experimentID,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint
0,i-002f7e27ea3e0bc1c,1573973929-0,m5d.large,60.237055,10,10,1,TPC-B (sort of),simple,60 s,123924,data_directory = '/var/lib/postgresql/9.5/main'
1,i-002f7e27ea3e0bc1c,1573973929-0,m5d.large,60.270962,10,10,1,TPC-B (sort of),simple,60 s,123006,data_directory = '/var/lib/postgresql/9.5/main'
2,i-002f7e27ea3e0bc1c,1573973929-0,m5d.large,60.281807,10,10,1,TPC-B (sort of),simple,60 s,122928,data_directory = '/var/lib/postgresql/9.5/main'
3,i-002f7e27ea3e0bc1c,1573973929-1,m5d.large,60.225921,10,10,1,TPC-B (sort of),simple,60 s,123065,data_directory = '/var/lib/postgresql/9.5/main'
4,i-002f7e27ea3e0bc1c,1573973929-1,m5d.large,60.268842,10,10,1,TPC-B (sort of),simple,60 s,123022,data_directory = '/var/lib/postgresql/9.5/main'


In [123]:
#setup cols, metric for the relative experiment type
def setUpMetrics(experimentName):
    
    if experimentName == 'pgbench':
        cols = ['instanceID', 'setId', 'transactions']
        metric = 'transactions'
    elif experimentName == 'sysbench' or experimentName == 'y-cruncher':
        cols = ['instanceID','setId','wallTime']
        metric = 'wallTime'
    elif experimentName == 'iperf':
        df_data['Total'] = df_data['Upload'] + df_data['Download']
        cols = ['instanceID','setId','Upload', 'Download', 'Total']
        metric = 'Total'
    else:
        print('Illegal Experiment Name..')
    return metric, cols

In [124]:
#setup vars for each experiment time... i.e. wallTime vs transactions
experimentName = 'pgbench'
metric, cols = setUpMetrics(experimentName)

In [125]:
#create setId column
df_data['setId'] = df_data['experimentID'].str.strip().str[-2:]
df_data['setId'] = df_data['setId'].astype('int32').abs()

In [126]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 13 columns):
instanceID          3528 non-null object
experimentID        3528 non-null object
instanceType        3528 non-null object
wallTime            3528 non-null float64
clients             3528 non-null int64
threads             3528 non-null int64
scaleFactor         3528 non-null int64
transactionsType    3528 non-null object
queryMode           3528 non-null object
duration            3528 non-null object
transactions        3528 non-null int64
mountPoint          3528 non-null object
setId               3528 non-null int32
dtypes: float64(1), int32(1), int64(4), object(7)
memory usage: 344.6+ KB


In [127]:
df_data.tail()

,instanceID,experimentID,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint,setId
3523,i-08e70b514c8c3cd5a,1573973929-45,m5d.large,60.289414,10,10,1,TPC-B (sort of),simple,60 s,171590,data_directory = '/var/lib/postgresql/9.5/main',45
3524,i-08e70b514c8c3cd5a,1573973929-45,m5d.large,60.211516,10,10,1,TPC-B (sort of),simple,60 s,171879,data_directory = '/var/lib/postgresql/9.5/main',45
3525,i-08e70b514c8c3cd5a,1573973929-46,m5d.large,60.288458,10,10,1,TPC-B (sort of),simple,60 s,180139,data_directory = '/var/lib/postgresql/9.5/main',46
3526,i-08e70b514c8c3cd5a,1573973929-46,m5d.large,60.199357,10,10,1,TPC-B (sort of),simple,60 s,182329,data_directory = '/var/lib/postgresql/9.5/main',46
3527,i-08e70b514c8c3cd5a,1573973929-46,m5d.large,60.207249,10,10,1,TPC-B (sort of),simple,60 s,184283,data_directory = '/var/lib/postgresql/9.5/main',46


In [128]:
#drop other columns
df_data = df_data[cols]

In [129]:
df_data.head()

,instanceID,setId,transactions
0,i-002f7e27ea3e0bc1c,0,123924
1,i-002f7e27ea3e0bc1c,0,123006
2,i-002f7e27ea3e0bc1c,0,122928
3,i-002f7e27ea3e0bc1c,1,123065
4,i-002f7e27ea3e0bc1c,1,123022


In [130]:
df_data.describe()

,setId,transactions
count,3528.000000,3528.000000
mean,15.666667,143490.297619
std,11.427711,15952.650512
min,0.000000,114134.000000
25%,6.000000,128668.750000
50%,14.000000,138478.000000
75%,24.000000,158502.500000
max,47.000000,184283.000000


In [131]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 3 columns):
instanceID      3528 non-null object
setId           3528 non-null int32
transactions    3528 non-null int64
dtypes: int32(1), int64(1), object(1)
memory usage: 69.0+ KB


In [132]:
#group by tenants / setId 
mean = df_data.groupby('setId').mean()
mean.head()

,transactions
setId,
0,123217.833333
1,123718.496454
2,124828.717391
3,126609.770370
4,127365.515152


In [133]:
#create result dataframe
result_df = mean

In [134]:
#create std, normalized data
std = df_data.groupby('setId').std()
std.head()

,transactions
setId,
0,1669.366954
1,1676.045011
2,1934.554872
3,1650.860042
4,1879.833473


In [135]:
#add normalized column
if experimentName == 'pgbench':
    result_df['normalized'] = (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100
    result_df['normalized'] += 100
else: 
    result_df['normalized'] = 1 - (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100

result_df['normalized'] = result_df['normalized'].round(2).astype(str) + '%'

In [136]:
result_df.head()

,transactions,normalized
setId,,
0,123217.833333,68.78%
1,123718.496454,69.06%
2,124828.717391,69.68%
3,126609.770370,70.67%
4,127365.515152,71.1%


In [137]:
#add std column
result_df['std'] = std

In [138]:
#reverse table
result_df=result_df.iloc[::-1]

In [139]:
#add tenants
result_df['tenants'] = np.arange(len(result_df))+1

/Users/dell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [140]:
#rearrange columns
cols = ['tenants', metric, 'std', 'normalized']

In [141]:
result_df = result_df[cols]

In [142]:
#add % to normalized

In [143]:
result_df

,tenants,transactions,std,normalized
setId,,,,
47,1,179145.333333,520.497198,100.0%
46,2,180675.000000,2173.306789,100.85%
45,3,172304.222222,1776.880608,96.18%
44,4,174785.583333,4665.384296,97.57%
43,5,176469.600000,4714.501319,98.51%
42,6,175958.722222,4513.089744,98.22%
41,7,173131.904762,1492.999427,96.64%
40,8,172668.875000,1735.253754,96.38%
39,9,169857.333333,3273.011716,94.82%


In [144]:
#print to csv
result_df.to_csv( path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [ ]:
# graph tenants vs wallTime